In [11]:
import pandas as pd
import os
import glob
import yfinance as yf
from datetime import datetime

In [ ]:
## PARA DESCARGAR LA NUEVA INFORMACION - TARDÓ 120 Minutos
"""# Ruta al directorio donde se encuentran los archivos CSV
directorio = 'C:/Users/Leidy Hernandez/Desktop/Soy Henry/Proyecto integrador/fh_20181104/full_history'

# Patrón para buscar archivos CSV
patron = '*.csv'

# Obtener la ruta de directorio de cada uno
archivos_csv = glob.glob(os.path.join(directorio, patron))

# Descargar los datos de Yahoo Finance y almacenarlos en un DataFrame por cada stock
for archivo_csv in archivos_csv:
    # Obtener el símbolo del stock desde el nombre del archivo
    nombre_archivo = os.path.basename(archivo_csv)
    simbolo = nombre_archivo.split('.')[0]  # Suponemos que el nombre del archivo es el símbolo del stock
    
    # Descargar los datos desde Yahoo Finance para los últimos 10 años
    data = yf.download(simbolo, start='2013-01-01')
    
    # Filtrar las columnas necesarias (Precio de apertura, cierre, cierre ajustado y volumen)
    data = data[['Open', 'Close', 'Adj Close', 'Volume']]
    
    # Guardar los datos en un nuevo archivo CSV
    nuevo_nombre_archivo = f'{simbolo}.csv'
    ruta_nuevo_archivo = os.path.join(directorio, nuevo_nombre_archivo)
    data.to_csv(ruta_nuevo_archivo)
    
    print(f"Se descargaron los datos para el stock {simbolo} y se guardaron en {nuevo_nombre_archivo}.") """

In [2]:
## PARA EXTRAER LOS NOMBRES DE LOS DOCUMENTOS 
# Ruta al directorio donde se encuentran los archivos CSV
directorio = 'C:/Users/Leidy Hernandez/Desktop/Soy Henry/Proyecto integrador/fh_20181104/full_history'

# Patrón para buscar archivos CSV
patron = '*.csv'

# Obtener la ruta de directorio de cada uno
archivos_csv = glob.glob(os.path.join(directorio, patron))

# Lista para almacenar los nombres de los archivos CSV
nombres_archivos_csv = []

# Extraer el nombre de cada archivo CSV
for archivo in archivos_csv:
    nombres_archivos_csv.append(os.path.basename(archivo))

In [3]:
## PARA CREAR LOS DATAFRAME
# Lista para almacenar los DataFrames de los archivos CSV
dataframes_csv = []
# Leer cada archivo CSV y almacenar el DataFrame correspondiente
for i in range(len(nombres_archivos_csv)):
    ruta_archivo = directorio + '/' + nombres_archivos_csv[i]  # Ruta completa al archivo
    df = pd.read_csv(ruta_archivo)
    dataframes_csv.append(df)

# Crea el DF por accion
for i in range(len(nombres_archivos_csv)):
    # Extraer un nombre de la lista
    nombre_variable = nombres_archivos_csv[i].replace('.csv','')
    # Asignar un valor a la variable con el nombre extraído
    globals()[nombre_variable] = dataframes_csv[i]

In [4]:
# Extraer sólo el nombre del stock
for i in range(len(nombres_archivos_csv)):
    nombres_archivos_csv[i] = nombres_archivos_csv[i].replace('.csv','')

In [5]:
## PARA ELIMINAR LOS STOCKS INÚTILES
archivos_inutiles = []
length = len(globals()[nombres_archivos_csv[0]]['Date'])

# Iterar sobre los nombres de archivos CSV
for nombre_archivo in nombres_archivos_csv:
    dataframe = globals()[nombre_archivo]
    fecha = dataframe['Date']
    
    # Se eliminan los df que no tengan datos, que no tengan histórico desde el primer día hábil del 2013,
    # que no estén cotizando para el día hábil más cercano de cuando descargamos los datos,
    # y que tengan información continua (que no falte ninguna cotización)
    if len(fecha) == 0 or fecha[0] != '2013-01-02' or fecha[len(fecha) - 1] != '2023-07-17' or len(fecha) != length:
        archivos_inutiles.append(nombre_archivo)

# Eliminar los DataFrames inútiles y sus nombres de archivos CSV
for nombre_archivo in archivos_inutiles:
    nombre_dataframe = nombre_archivo.replace('.csv', '')
    del globals()[nombre_dataframe]
    nombres_archivos_csv.remove(nombre_archivo)

In [6]:
## CALCULAR DESVIACIÓN ESTANDARD
import numpy as np
metrics = pd.DataFrame()

metrics['Stock'] = nombres_archivos_csv
std = []

for i in range(len(metrics['Stock'])):
    std.append(np.std(globals()[metrics['Stock'][i]]['Adj Close']))

metrics['Standard Deviation'] = std


In [7]:
## RANGO DE PRECIOS
r_min = []
r_max = []
fecha = '2023-07-14'    ## Fecha de 'Ayer', del dia anterior para basarse ena estimación del precio de "hoy" (Siempre tiene que ser de días hábiles)
indice = globals()[metrics['Stock'][0]].loc[A['Date'] == fecha].index[0]    ## Como todos los archivos tienen la misma cantidad de datos, si en uno se encuentra la fecha en x 
                                                                            ## posicion entonces en esa misma estará para todos los demás df
for i in range(len(metrics['Stock'])):
    r_min.append(globals()[metrics['Stock'][i]]['Close'][indice] - metrics['Standard Deviation'][i])
    r_max.append(globals()[metrics['Stock'][i]]['Close'][indice] + metrics['Standard Deviation'][i])

metrics['Rango Mínimo Estimado'] = r_min
metrics['Rango Máximo Estimado'] = r_max

In [8]:
## PARA CALCULAR LOS RENDIMIENTOS -> (Pf/Pi)-1
for j in range(len(nombres_archivos_csv)):
    nombre_archivo = nombres_archivos_csv[j]
    dataframe = globals()[nombre_archivo]
    dataframe['Rendimientos (%)'] = np.nan
    adj_close = dataframe['Adj Close']
    rendimientos = [((adj_close[i] / adj_close[i-1]) - 1)*100 for i in range(1, len(adj_close))]
    dataframe['Rendimientos (%)'][1:] = rendimientos    

C:\Users\Leidy Hernandez\AppData\Local\Temp\ipykernel_1092\1063927095.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['Rendimientos (%)'][1:] = rendimientos
C:\Users\Leidy Hernandez\AppData\Local\Temp\ipykernel_1092\1063927095.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['Rendimientos (%)'][1:] = rendimientos
C:\Users\Leidy Hernandez\AppData\Local\Temp\ipykernel_1092\1063927095.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returnin

In [12]:
A.head()

,Date,Open,Close,Adj Close,Volume,Rendimientos (%)
0,2013-01-02,30.164520,29.957083,27.338415,8790205,NaN
1,2013-01-03,30.000000,30.064379,27.436342,5751791,0.358203
2,2013-01-04,30.143063,30.658083,27.978144,6432897,1.974758
3,2013-01-07,30.472103,30.436338,27.775780,3589505,-0.723293
4,2013-01-08,30.379112,30.193132,27.553833,3896925,-0.799066


In [13]:
## CALCULAR PROMEDIO DE RENDIMIENTOS DIARIOS POR STOCK
p_rend = []
for i in range(len(metrics['Stock'])):
    p_rend.append(np.mean(globals()[metrics['Stock'][i]]['Rendimientos (%)']))
metrics['Rendimiento promedio desde 2013-01-01 (%)'] = p_rend

In [15]:
sorted(metrics['Rendimiento promedio desde 2013-01-01 (%)'], reverse=True)

[7.823847768203877,
 1.8958787035565257,
 1.4794848995237952,
 1.321361581817079,
 1.0401272663150813,
 0.6194196906559009,
 0.5812580228458641,
 0.45887614800222215,
 0.4314833778213016,
 0.39606501362958824,
 0.39334702181770975,
 0.34496343698405263,
 0.3424794430730984,
 0.3218911799497871,
 0.32066608115030865,
 0.3040225111904476,
 0.3033754657600857,
 0.302357060243553,
 0.29532449881601236,
 0.27730903129387074,
 0.2760937039108245,
 0.2724612103451301,
 0.27060175876423825,
 0.26342142967121035,
 0.26265341137997883,
 0.2607084816919331,
 0.25635771220843123,
 0.2529144333895145,
 0.2523287092446384,
 0.25132083385883597,
 0.24987245489858817,
 0.2460619176176027,
 0.24532767751997078,
 0.24261136233922936,
 0.24112131429060357,
 0.23694681263963688,
 0.2344473211341396,
 0.23339645805981968,
 0.2326961402663566,
 0.22908796715882324,
 0.22675566286526425,
 0.22541491569386768,
 0.22502848434227507,
 0.21746838231534227,
 0.21568530609541384,
 0.2131285802145555,
 0.2124615065

In [16]:
metrics

,Stock,Standard Deviation,Rango Mínimo Estimado,Rango Máximo Estimado,Rendimiento promedio desde 2013-01-01 (%)
0,A,40.370252,79.379748,160.120252,0.068761
1,AA,14.019552,20.500449,48.539552,0.070078
2,AAL,12.522921,5.617078,30.662921,0.060808
3,AAME,0.750529,1.069471,2.570529,0.067199
4,AAON,21.281123,78.168874,120.731120,0.115070
...,...,...,...,...,...
2765,ZEUS,8.639032,43.280966,60.559030,0.095303
2766,ZG,29.034645,23.355355,81.424644,0.125342
2767,ZION,12.433118,18.266882,43.133119,0.046749
2768,ZTR,1.082080,5.017920,7.182080,0.026816


In [125]:
((29.564398/27.338415)-1)*100

8.142326466256367

In [17]:
A

,Date,Open,Close,Adj Close,Volume,Rendimientos (%)
0,2013-01-02,30.164520,29.957083,27.338415,8790205,NaN
1,2013-01-03,30.000000,30.064379,27.436342,5751791,0.358203
2,2013-01-04,30.143063,30.658083,27.978144,6432897,1.974758
3,2013-01-07,30.472103,30.436338,27.775780,3589505,-0.723293
4,2013-01-08,30.379112,30.193132,27.553833,3896925,-0.799066
...,...,...,...,...,...,...
2647,2023-07-11,118.300003,118.480003,118.480003,1096700,0.101388
2648,2023-07-12,120.000000,120.000000,120.000000,1595400,1.282914
2649,2023-07-13,120.760002,119.349998,119.349998,1229800,-0.541668
2650,2023-07-14,119.540001,119.750000,119.750000,1225400,0.335150
